INSTALL THE FOLLOWING PYTHON PACKAGES FIRST BEFORE RUNNING THE PROGRAM

1) Numpy
2) NNFS - for the Spiral dataset
3) scikit-learn - for the iris dataset

In [43]:
# Library imports
import numpy as np

Create classes for modularity

In [44]:
# Hidden Layers - Dense
class Layer_Dense:
    # Layer initialization
    def __init__(self, n_inputs, n_neurons):
        # IMPROVED: He Initialization for better accuracy with ReLU
        self.weights = np.random.randn(n_inputs, n_neurons) * np.sqrt(2. / n_inputs)
        self.biases = np.zeros((1, n_neurons))

    # Forward pass
    def forward(self, inputs):
        self.inputs = inputs
        self.output = np.dot(inputs, self.weights) + self.biases

    # Backward pass
    def backward(self, dvalues):
        self.dweights = np.dot(self.inputs.T, dvalues)
        self.dbiases = np.sum(dvalues, axis=0, keepdims=True)
        self.dinputs = np.dot(dvalues, self.weights.T)

In [45]:
# Activation Functions
# Included here are the functions for both the forward and backward pass

# Linear
class ActivationLinear:
    def forward(self, inputs):
        self.inputs = inputs
        self.output = inputs

    def backward(self, dvalues):
        self.dinputs = dvalues.copy()

# Sigmoid
class ActivationSigmoid:
    def forward(self, inputs):
        self.inputs = inputs
        self.output = 1 / (1 + np.exp(-inputs))

    def backward(self, dvalues):
        self.dinputs = dvalues * (self.output * (1 - self.output))

# TanH
class ActivationTanH:
    def forward(self, inputs):
        self.inputs = inputs
        self.output = np.tanh(inputs)

    def backward(self, dvalues):
        self.dinputs = dvalues * (1 - self.output ** 2)

# ReLU
class Activation_ReLU:
    # Forward pass
    def forward(self, inputs):
        # Remember the input values
        self.inputs = inputs
        # Calculate the output values from inputs
        self.output = np.maximum(0, inputs)

    # Backward pass
    def backward(self, dvalues):
        # Make a copy of the original values first
        self.dinputs = dvalues.copy()
    
        # Zero gradient where input values were negative
        self.dinputs[self.inputs <= 0] = 0

# Softmax
class Activation_Softmax:
    # Forward pass
    def forward(self, inputs):
        # Remember the inputs values
        self.inputs = inputs

        # Get the unnormalized probabilities
        exp_values = np.exp(inputs - np.max(inputs, axis=1, keepdims=True))

        # Normalize them for each sample
        probabilities = exp_values / np.sum(exp_values, axis=1, keepdims=True)

        self.output = probabilities

    # Backward pass
    def backward(self, dvalues):
        # Create uninitialized array
        self.dinputs = np.empty_like(dvalues)

        # Enumerate outputs and gradients
        for index, (single_output, single_dvalues) in enumerate(zip(self.output, dvalues)):

            # Flatten output array
            single_output = single_output.reshape(-1, 1)
            # Calculate Jacobian matrix of the output
            jacobian_matrix = np.diagflat(single_output) - np.dot(single_output, single_output.T)
            # Calculate the sample-wise gradient
            # and add it to the array of sample gradients
            self.dinputs[index] = np.dot(jacobian_matrix, single_dvalues)

In [46]:
# Loss functions

class Loss:
    # Calculate the data and regularization losses
    # Given the model output and grou truth/target values
    def calculate(self, output, y):
        # Calculate sample losses
        sample_losses = self.forward(output, y)
        # Calculate the mean loss
        data_loss = np.mean(sample_losses)
        # Return the mean loss
        return data_loss

# MSE
class Loss_MSE:
    def forward(self, y_pred, y_true):
        # Calculate Mean Squared Error
        return np.mean((y_true - y_pred) ** 2, axis=-1)

    def backward(self, y_pred, y_true):
        # Gradient of MSE loss
        samples = y_true.shape[0]
        outputs = y_true.shape[1]
        self.dinputs = -2 * (y_true - y_pred) / outputs
        # Normalize gradients over samples
        self.dinputs = self.dinputs / samples

# Binary Cross-Entropy
class Loss_BinaryCrossEntropy:
    def forward(self, y_pred, y_true):
        # Clip predictions
        y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)
        # Calculate Binary Cross Entropy
        return -(y_true * np.log(y_pred_clipped) + (1 - y_true) * np.log(1 - y_pred_clipped))

    def backward(self, dvalues, y_true):
        samples = len(dvalues)
        labels = len(dvalues[0])

        if len(y_true.shape) == 1:
            y_true = np.eye(labels)[y_true]

        # IMPROVED: Added 1e-7 to prevent division by zero (nan)
        self.dinputs = -y_true / (dvalues + 1e-7)
        self.dinputs = self.dinputs / samples

# Categorical Cross-Entropy
class Loss_CategoricalCrossEntropy(Loss):
    # (Keep your forward method same as original)
    def forward(self, y_pred, y_true):
        samples = y_pred.shape[0]
        y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)
        if len(y_true.shape) == 1:
            correct_confidences = y_pred_clipped[range(samples), y_true]
        elif len(y_true.shape) == 2:
            correct_confidences = np.sum(y_pred_clipped * y_true, axis=1)
        negative_log_likelihoods = -np.log(correct_confidences)
        return negative_log_likelihoods

    # Backward pass (Stabilized)
    def backward(self, dvalues, y_true):
        samples = len(dvalues)
        labels = len(dvalues[0])
        if len(y_true.shape) == 1:
            y_true = np.eye(labels)[y_true]

        # IMPROVED: Added 1e-7 to prevent division by zero/nan
        self.dinputs = -y_true / (dvalues + 1e-7)
        self.dinputs = self.dinputs / samples


<!-- Star -->

In [47]:
# Updated Optimizer with three variants
class Optimizer_SGD:
    def __init__(self, learning_rate=1.0, decay=0., momentum=0., use_adagrad=False):
        self.learning_rate = learning_rate
        self.current_learning_rate = learning_rate
        self.decay = decay
        self.iterations = 0
        self.momentum = momentum
        self.use_adagrad = use_adagrad
    
    # HINT: Learning decay happens BEFORE FP and BP
    def pre_update_params(self):
        if self.decay:
            self.current_learning_rate = self.learning_rate * (1. / (1. + self.decay * self.iterations))

    # HINT: Momentum/Vanilla/AdaGrad logic happens AFTER decay
    def update_params(self, layer):
        # 1. AdaGrad logic (Stabilized with epsilon to prevent nan)
        if self.use_adagrad:
            if not hasattr(layer, 'weight_cache'):
                layer.weight_cache = np.zeros_like(layer.weights)
                layer.bias_cache = np.zeros_like(layer.biases)
            layer.weight_cache += layer.dweights**2
            layer.bias_cache += layer.dbiases**2
            layer.weights += -self.current_learning_rate * layer.dweights / (np.sqrt(layer.weight_cache) + 1e-7)
            layer.biases += -self.current_learning_rate * layer.dbiases / (np.sqrt(layer.bias_cache) + 1e-7)
        
        # 2. SGD with Momentum logic
        elif self.momentum:
            if not hasattr(layer, 'weight_momentums'):
                layer.weight_momentums = np.zeros_like(layer.weights)
                layer.bias_momentums = np.zeros_like(layer.biases)
            weight_updates = self.momentum * layer.weight_momentums - self.current_learning_rate * layer.dweights
            layer.weight_momentums = weight_updates
            bias_updates = self.momentum * layer.bias_momentums - self.current_learning_rate * layer.dbiases
            layer.bias_momentums = bias_updates
            layer.weights += weight_updates
            layer.biases += bias_updates
            
        # 3. Vanilla SGD logic
        else:
            layer.weights += -self.current_learning_rate * layer.dweights
            layer.biases += -self.current_learning_rate * layer.dbiases

    def post_update_params(self):
        self.iterations += 1

Use most of the classes to create a functioning neural network, capable of performing a forward and backward pass

We can use a sample dataset from the Spiral module.  

We can also use the IRIS dataset.

In [48]:
# Spiral Data
import nnfs
from nnfs.datasets import spiral_data

# Create the dataset
X, y = spiral_data(samples = 100, classes = 3)

# print(X[:5])
# print(X.shape)
# print(y[:5])
# print(y.shape)

In [49]:
# Iris Dataset
# From the scikit-learn library
# from sklearn.datasets import load_iris
# iris = load_iris()
# X = iris.data # Features
# y = iris.target # Target labels

# print(X[:5])
# print(X.shape)
# print(y[:5])
# print(y.shape)

In [50]:
# Neural Network initialization
# Create a Dense Layer with 2 input features and 3 output values
dense1 = Layer_Dense(2, 3)

# Make sure you check the shape of the features, in order to adjust the input size of the first layer
# dense1 = Layer_Dense(4, 3)

# Create a ReLU activation for the first Dense layer
activation1 = Activation_ReLU()

# Create a 2nd dense layer with 3 input and 3 output values
dense2 = Layer_Dense(3, 3)

# Create a Softmax activation for the 2nd Dense layer
activation2 = Activation_Softmax()

# Create a loss function
loss_function = Loss_CategoricalCrossEntropy()

# Create the optimizer
optimizer = Optimizer_SGD()

PERFORM ONLY 1 PASS

In [51]:
# Perform a forward pass of our training data
# give the input from the dataset to the first layer
dense1.forward(X)

# Activation function
activation1.forward(dense1.output)

# Pass on the 2nd layer
dense2.forward(activation1.output)

activation2.forward(dense2.output)

# Calculate the loss
loss_function.forward(activation2.output, y)

# Check the model's performance
predictions = np.argmax(activation2.output, axis=1)
if len(y.shape) == 2:
    y = np.argmax(y, axis=1)
accuracy = np.mean(predictions == y)

# Print the accuracy
print('acc:', accuracy)

acc: 0.3566666666666667


In [52]:
# Perform a backward pass of our training data
# From loss to 2nd softmax activation
loss_function.backward(activation2.output, y)
dvalues = loss_function.dinputs # Gradient of the loss w.r.t softmax output

print(dvalues.shape)
# print(dvalues)

# From 2nd softmax to 2nd dense layer
activation2.backward(dvalues)
# From 2nd dense layer to 1st ReLU activation
dense2.backward(activation2.dinputs)

# From 1st ReLU activation to 1st dense layer
activation1.backward(dense2.dinputs)
dense1.backward(activation1.dinputs)

(300, 3)


In [53]:
# Check the gradient values of the weights and biases of the established layers
print(dense1.dweights)
print(dense1.dbiases)
print(dense2.dweights)
print(dense2.dbiases)


# Update the weights and biases
optimizer.update_params(dense1)
optimizer.update_params(dense2)

[[ 0.00134027  0.0668231   0.0464467 ]
 [ 0.00347656  0.02707032 -0.00381398]]
[[-1.04299015e-04  1.92093228e-01  9.70856086e-02]]
[[ 0.00830788  0.00385047 -0.01215835]
 [-0.07546864  0.05670387  0.01876478]
 [-0.10086348  0.10229807 -0.00143459]]
[[-0.0991977   0.09041487  0.00878283]]


In [54]:
# Hidden Layers - Dense
class Layer_Dense:
    # Layer initialization
    def __init__(self, n_inputs, n_neurons):
        # IMPROVED: He Initialization (np.sqrt(2/n_inputs))
        # This replaces the old 0.01 * np.random.randn logic
        self.weights = np.random.randn(n_inputs, n_neurons) * np.sqrt(2. / n_inputs)
        self.biases = np.zeros((1, n_neurons))

    # Forward pass
    def forward(self, inputs):
        self.inputs = inputs
        self.output = np.dot(inputs, self.weights) + self.biases

    # Backward pass
    def backward(self, dvalues):
        self.dweights = np.dot(self.inputs.T, dvalues)
        self.dbiases = np.sum(dvalues, axis=0, keepdims=True)
        self.dinputs = np.dot(dvalues, self.weights.T)

In [55]:
# Activation Functions
class Activation_ReLU:
    def forward(self, inputs):
        self.inputs = inputs
        self.output = np.maximum(0, inputs)

    def backward(self, dvalues):
        self.dinputs = dvalues.copy()
        self.dinputs[self.inputs <= 0] = 0

class Activation_Softmax:
    def forward(self, inputs):
        self.inputs = inputs
        exp_values = np.exp(inputs - np.max(inputs, axis=1, keepdims=True))
        probabilities = exp_values / np.sum(exp_values, axis=1, keepdims=True)
        self.output = probabilities

    def backward(self, dvalues):
        self.dinputs = np.empty_like(dvalues)
        for index, (single_output, single_dvalues) in enumerate(zip(self.output, dvalues)):
            single_output = single_output.reshape(-1, 1)
            jacobian_matrix = np.diagflat(single_output) - np.dot(single_output, single_output.T)
            self.dinputs[index] = np.dot(jacobian_matrix, single_dvalues)


In [56]:
# Loss functions

class Loss:
    # Calculate the data and regularization losses
    # Given the model output and grou truth/target values
    def calculate(self, output, y):
        # Calculate sample losses
        sample_losses = self.forward(output, y)
        # Calculate the mean loss
        data_loss = np.mean(sample_losses)
        # Return the mean loss
        return data_loss

# MSE
class Loss_MSE:
    def forward(self, y_pred, y_true):
        # Calculate Mean Squared Error
        return np.mean((y_true - y_pred) ** 2, axis=-1)

    def backward(self, y_pred, y_true):
        # Gradient of MSE loss
        samples = y_true.shape[0]
        outputs = y_true.shape[1]
        self.dinputs = -2 * (y_true - y_pred) / outputs
        # Normalize gradients over samples
        self.dinputs = self.dinputs / samples

# Binary Cross-Entropy
class Loss_BinaryCrossEntropy:
    def forward(self, y_pred, y_true):
        # Clip predictions
        y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)
        # Calculate Binary Cross Entropy
        return -(y_true * np.log(y_pred_clipped) + (1 - y_true) * np.log(1 - y_pred_clipped))

    def backward(self, y_pred, y_true):
        # Gradient of BCE loss
        samples = y_true.shape[0]
        y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)
        self.dinputs = - (y_true / y_pred_clipped - (1 - y_true) / (1 - y_pred_clipped))
        # Normalize gradients over samples
        self.dinputs = self.dinputs / samples

# Categorical Cross-Entropy
class Loss_CategoricalCrossEntropy(Loss):
    # Forward pass
    def forward(self, y_pred, y_true):
        # Number of samples in a batch
        samples = y_pred.shape[0]

        # Clip data to prevent division by 0
        # Clip both sides to not drag mean towards any value
        y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)

        # Probabilities for target values
        # Only if categorical labels
        if len(y_true.shape) == 1:
            correct_confidences = y_pred_clipped[range(samples), y_true]
        # Mask values - only for one-hot encoded labels
        elif len(y_true.shape) == 2:
            correct_confidences = np.sum(y_pred_clipped * y_true, axis=1)

        # Losses
        negative_log_likelihoods = -np.log(correct_confidences)
        return negative_log_likelihoods

    # Backward pass
    def backward(self, dvalues, y_true):
        # Number of samples
        samples = len(dvalues)
        # Number of labels in every sample
        # Use the first sample to count them
        labels = len(dvalues[0])

        # Check if labels are sparse, turn them into one-hot vector values
        # the eye function creates a 2D array with ones on the diagonal and zeros elsewhere
        if len(y_true.shape) == 1:
            y_true = np.eye(labels)[y_true]

        # Calculate the gradient
        self.dinputs = -y_true / dvalues
        self.dinputs = self.dinputs / samples



In [57]:
class Optimizer_SGD:
    def __init__(self, learning_rate=1.0, decay=0., momentum=0., use_adagrad=False):
        self.learning_rate = learning_rate
        self.current_learning_rate = learning_rate
        self.decay = decay
        self.iterations = 0
        self.momentum = momentum
        self.use_adagrad = use_adagrad
    
    # HINT: Learning decay happens BEFORE FP and BP
    def pre_update_params(self):
        if self.decay:
            self.current_learning_rate = self.learning_rate * (1. / (1. + self.decay * self.iterations))

    # HINT: Implementation of momentum/vanilla/adagrad happens AFTER decay
    def update_params(self, layer):
        # 1. AdaGrad logic (Stabilized with epsilon)
        if self.use_adagrad:
            if not hasattr(layer, 'weight_cache'):
                layer.weight_cache = np.zeros_like(layer.weights)
                layer.bias_cache = np.zeros_like(layer.biases)
            
            layer.weight_cache += layer.dweights**2
            layer.bias_cache += layer.dbiases**2
            
            layer.weights += -self.current_learning_rate * layer.dweights / (np.sqrt(layer.weight_cache) + 1e-7)
            layer.biases += -self.current_learning_rate * layer.dbiases / (np.sqrt(layer.bias_cache) + 1e-7)
        
        # 2. SGD with Momentum logic
        elif self.momentum:
            if not hasattr(layer, 'weight_momentums'):
                layer.weight_momentums = np.zeros_like(layer.weights)
                layer.bias_momentums = np.zeros_like(layer.biases)
            
            weight_updates = self.momentum * layer.weight_momentums - self.current_learning_rate * layer.dweights
            layer.weight_momentums = weight_updates
            
            bias_updates = self.momentum * layer.bias_momentums - self.current_learning_rate * layer.dbiases
            layer.bias_momentums = bias_updates
            
            layer.weights += weight_updates
            layer.biases += bias_updates
            
        # 3. Vanilla SGD logic
        else:
            layer.weights += -self.current_learning_rate * layer.dweights
            layer.biases += -self.current_learning_rate * layer.dbiases

    def post_update_params(self):
        self.iterations += 1

In [58]:
# Spiral Data
import nnfs
from nnfs.datasets import spiral_data

# Create the dataset
X, y = spiral_data(samples=100, classes=3)

print(X[:5])
print(X.shape)
print(y[:5])
print(y.shape)

[[-0.          0.        ]
 [ 0.00576021  0.00829761]
 [ 0.01075496  0.01710124]
 [ 0.02923636  0.00796925]
 [ 0.00809216  0.03958539]]
(300, 2)
[0 0 0 0 0]
(300,)


In [59]:
# Neural Network initialization
# Create a Dense Layer with 2 input features and 64 output values (IMPROVED)
dense1 = Layer_Dense(2, 64)

# Create a ReLU activation for the first Dense layer
activation1 = Activation_ReLU()

# Create a 2nd dense layer with 64 input and 3 output values (IMPROVED)
dense2 = Layer_Dense(64, 3)

# Create a Softmax activation for the 2nd Dense layer
activation2 = Activation_Softmax()

# Create a loss function
loss_function = Loss_CategoricalCrossEntropy()

# Create the optimizer (for single pass testing)
optimizer = Optimizer_SGD()

In [60]:
# Perform a forward pass of our training data
# give the input from the dataset to the first layer
dense1.forward(X)

# Activation function
activation1.forward(dense1.output)

# Pass on the 2nd layer
dense2.forward(activation1.output)

activation2.forward(dense2.output)

# Calculate the loss
loss = loss_function.calculate(activation2.output, y)

# Check the model's performance
predictions = np.argmax(activation2.output, axis=1)
if len(y.shape) == 2:
    y_labels = np.argmax(y, axis=1)
else:
    y_labels = y
accuracy = np.mean(predictions == y_labels)

# Print the accuracy
print('acc:', accuracy)


acc: 0.27666666666666667


In [61]:
# Perform a backward pass of our training data
# From loss to 2nd softmax activation
loss_function.backward(activation2.output, y)
dvalues = loss_function.dinputs # Gradient of the loss w.r.t softmax output

print(dvalues.shape)
# print(dvalues)

# From 2nd softmax to 2nd dense layer
activation2.backward(dvalues)
# From 2nd dense layer to 1st ReLU activation
dense2.backward(activation2.dinputs)

# From 1st ReLU activation to 1st dense layer
activation1.backward(dense2.dinputs)
dense1.backward(activation1.dinputs)


(300, 3)


In [62]:
# Check the gradient values of the weights and biases of the established layers
print(dense1.dweights)
print(dense1.dbiases)
print(dense2.dweights)
print(dense2.dbiases)

# Update the weights and biases
optimizer.update_params(dense1)
optimizer.update_params(dense2)

[[ 2.50020201e-03 -5.95990430e-03 -1.77545518e-02 -5.00678735e-03
  -1.31370933e-03 -4.51610446e-03 -3.50646635e-04  1.16396923e-02
  -6.09668051e-03 -1.54835612e-02  1.29724439e-03 -5.08519310e-03
  -5.62666925e-03 -2.12893028e-03  2.50352972e-03  2.82144404e-03
   3.84347701e-03 -7.24350723e-03 -5.82520759e-03 -1.94013379e-03
  -2.31093185e-04 -6.99621407e-03  3.43468562e-04 -2.19795481e-02
  -2.31936163e-03 -4.26888516e-03 -1.32628231e-03  5.07252455e-03
  -6.41821122e-03  2.73218760e-03 -3.29871453e-03 -1.98254192e-04
  -5.48040179e-03  2.34965753e-03  1.19534811e-03 -4.48305821e-03
   3.89279541e-04  1.41500665e-02 -1.76255405e-02 -7.11202302e-04
  -7.98716228e-03 -8.31467797e-03  1.70073529e-02 -6.71203526e-03
  -9.43008772e-04  8.06888123e-04 -4.43301100e-04  9.68137962e-04
  -4.17538039e-07 -1.15556930e-03  5.92539421e-04 -1.87224559e-03
  -6.70127107e-03  1.20624620e-02 -3.53326131e-03  2.26755317e-04
   1.79897948e-04  3.90556553e-03 -9.82103292e-03  2.62637398e-03
  -5.29648

In [63]:
def train_with_optimizer(optimizer_name, optimizer):
    # Re-initialize for fresh training with He weights
    dense1 = Layer_Dense(2, 64)
    activation1 = Activation_ReLU()
    dense2 = Layer_Dense(64, 3)
    activation2 = Activation_Softmax()
    loss_function = Loss_CategoricalCrossEntropy()
    
    print(f"\n{'='*70}\nTraining with: {optimizer_name}\n{'='*70}")
    
    for epoch in range(1001):
        # 1. HINT: Decay BEFORE FP/BP
        optimizer.pre_update_params()
        
        # 2. Forward Pass
        dense1.forward(X)
        activation1.forward(dense1.output)
        dense2.forward(activation1.output)
        activation2.forward(dense2.output)
        loss = loss_function.calculate(activation2.output, y)
        
        # Accuracy calculation
        predictions = np.argmax(activation2.output, axis=1)
        y_labels = np.argmax(y, axis=1) if len(y.shape) == 2 else y
        accuracy = np.mean(predictions == y_labels)
        
        # 3. Backward Pass
        loss_function.backward(activation2.output, y)
        activation2.backward(loss_function.dinputs)
        dense2.backward(activation2.dinputs)
        activation1.backward(dense2.dinputs)
        dense1.backward(activation1.dinputs)
        
        # 4. HINT: Updates AFTER decay
        optimizer.update_params(dense1)
        optimizer.update_params(dense2)
        optimizer.post_update_params()
        
        if epoch % 100 == 0:
            print(f'Epoch: {epoch}, Acc: {accuracy:.4f}, Loss: {loss:.4f}, LR: {optimizer.current_learning_rate:.6f}')
            
    return accuracy

# Final Calibrated Hyperparameters
acc1 = train_with_optimizer("Vanilla SGD + Decay", Optimizer_SGD(learning_rate=1.0, decay=1e-3))
acc2 = train_with_optimizer("SGD with Momentum", Optimizer_SGD(learning_rate=0.1, decay=1e-3, momentum=0.9))
acc3 = train_with_optimizer("AdaGrad", Optimizer_SGD(learning_rate=0.5, decay=1e-3, use_adagrad=True))


Training with: Vanilla SGD + Decay
Epoch: 0, Acc: 0.3067, Loss: 1.1583, LR: 1.000000
Epoch: 100, Acc: 0.4733, Loss: 1.0391, LR: 0.909091
Epoch: 200, Acc: 0.4867, Loss: 1.0296, LR: 0.833333
Epoch: 300, Acc: 0.4900, Loss: 1.0242, LR: 0.769231
Epoch: 400, Acc: 0.5100, Loss: 1.0131, LR: 0.714286
Epoch: 500, Acc: 0.5133, Loss: 0.9982, LR: 0.666667
Epoch: 600, Acc: 0.5333, Loss: 0.9801, LR: 0.625000
Epoch: 700, Acc: 0.5333, Loss: 0.9631, LR: 0.588235
Epoch: 800, Acc: 0.5367, Loss: 0.9456, LR: 0.555556
Epoch: 900, Acc: 0.5567, Loss: 0.9325, LR: 0.526316
Epoch: 1000, Acc: 0.5667, Loss: 0.9131, LR: 0.500000

Training with: SGD with Momentum
Epoch: 0, Acc: 0.3533, Loss: 1.1294, LR: 0.100000
Epoch: 100, Acc: 0.4833, Loss: 1.0391, LR: 0.090909
Epoch: 200, Acc: 0.4900, Loss: 1.0246, LR: 0.083333
Epoch: 300, Acc: 0.4900, Loss: 1.0099, LR: 0.076923
Epoch: 400, Acc: 0.4900, Loss: 0.9957, LR: 0.071429
Epoch: 500, Acc: 0.5000, Loss: 0.9832, LR: 0.066667
Epoch: 600, Acc: 0.5067, Loss: 0.9732, LR: 0.0625

In [ ]:
#AdaGrad stabilized the fastest, showing sharp loss reduction as early as Epoch 100 and maintaining a smooth, consistent decline throughout the training. 
#Vanilla SGD showed moderate stabilization but began to stagnate after Epoch 600, with the loss curve flattening out significantly. 
#SGD with Momentum was the slowest to stabilize; due to its lower initial learning rate, it did not show meaningful progress until after Epoch 400 and remained the least stable by the end of the run.

#AdaGrad achieved the highest accuracy at 80.00%, as its adaptive learning rate effectively mapped the complex, non-linear patterns of the spiral dataset.
#Vanilla SGD reached 56.67%, outperforming Momentum because its higher starting learning rate allowed for more aggressive initial convergence.
#SGD with Momentum finished with the lowest accuracy at 52.00%, as it lacked the necessary velocity to overcome its conservative step size within the 1000-epoch limit.